In [1]:
import torch
import torch.nn as nn
import os
import pandas as pd
from data_parsing import process_file

from tqdm import tqdm

print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 1.13.1+cu117


In [10]:
%run successive_shortest_paths.ipynb

number of edges: 24
Number of flow updates: 5, final flow value: 150


/home/alexandre/miniconda3/envs/cs224w/lib/python3.10/site-packages/nbformat/__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


ValueError: too many values to unpack (expected 2)

ValueError: too many values to unpack (expected 2)

In [11]:
# def parse(filename) -> Network:
#     """
#     Parses a network file following the DIMACS problem specification
#     structure and transforms it into a Network object

#     Some elements of the specification:
#     - Lines starting in c are comments
#     - Lines starting in p explain what problem to solve (can be ignored,
#       we only consider minimum-cost flow problems)
#     - Lines starting in n define nodes
#     - Lines starting in a define arcs (edges)

#     Args:
#         filename: name of the file containing the network data

#     Returns:
#         The corresponding Network object
#     """
#     # Lines we can ignore
#     ignore_list = ['c', 'p']

#     file = open(filename, 'r')

#     # Nodes is a hashmap from node values to their supply
#     nodes = {}
#     # Edges is a hashmap from edges to a tuple with their capacity and cost
#     edges = {}

#     for line in file:
#         if len(line) > 0 and line[0] not in ignore_list:
#             if line[0] == 'n':
#                 # Node parsing
#                 node = [int(elem) for elem in line.split(' ')[1:]]
#                 nodes[node[0]] = node[1]
#             elif line[0] == 'a':
#                 arc = [int(elem) for elem in line.split(' ')[1:]]
#                 node1 = arc[0]
#                 node2 = arc[1]
#                 capacity = arc[3]
#                 cost = arc[4]

#                 # Only nodes with non-zero supply are in a "node line"
#                 if node1 not in nodes:
#                     nodes[node1] = 0
#                 if node2 not in nodes:
#                     nodes[node2] = 0
#                 if (node1, node2) in edges:
#                     # TODO not amazing (reaverages every time)
#                     old_capacity, old_cost = edges[(node1, node2)]
#                     new_cost = old_cost * old_capacity + cost * capacity
#                     new_cost /= (old_capacity + capacity)
#                     edges[(node1, node2)] = (old_capacity + capacity, new_cost)
#                 else:
#                     edges[(node1, node2)] = (capacity, cost)
#     file.close()

#     capacities, costs = zip(*edges.values())
#     network = Network(list(nodes.keys()), list(edges.keys()), capacities, costs, list(nodes.values()))
#     #TODO data types?
#     print(f"This dataset contains: {len(nodes.keys())} nodes and {len(edges.keys())} edges")
#     if len(edges.keys()) <= 1e6:
#         index = {node: index for node, index in zip(nodes, range(len(nodes)))}
#         x = torch.tensor([supply for supply in nodes.values()])
#         edge_index = torch.reshape(torch.tensor([[index[e[0]], index[e[1]]] for e in edges]), (2, -1))
#         edge_attr = torch.reshape(torch.tensor([list(attributes) for attributes in edges.values()]), (2, -1))
#         print("starting to run successive shortest paths")
#         iter_limit = 150
#         converged, f, p = successive_shortest_paths(network, iter_limit = iter_limit)
#         print("finished running successive shortest paths")
#         y = dual_value(network, p)
#         if converged:
#             return {"converged": True, "x": x, "edge_index": edge_index, "edge_attr": edge_attr, "y": y}
#     return {"converged": False}

In [38]:
import os.path as osp

import torch
from torch_geometric.data import Data, Dataset, download_url


class MinCostDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        """If these files are found in the raw directory, download is skipped"""
        return []

    @property
    def processed_file_names(self):
        """If these files are found in the processed directory, processing is skipped"""
        processed_files = []
        path = self.processed_dir
        for file in tqdm(os.listdir(path)):
            file_path = os.path.join(path, file)
            if not os.path.isdir(file_path) and not file == "pre_filter.pt" and not file == "pre_transform.pt":
                processed_files.append(file)

        return processed_files

    def download(self):
        pass

    def process(self):
        idx = 32
        path = self.raw_dir
        for file in tqdm(os.listdir(path)):
            print(file)
            file_path = os.path.join(path, file)
            if os.path.isdir(file_path):
                continue
            # Read data from `raw_path`.
            output = process_file(file_path, 'nx')
            if output["converged"]:
                x = output["x"].type(torch.FloatTensor)
                edge_index = output["edge_index"]
                edge_attr = output["edge_attr"].type(torch.FloatTensor)
                y = output["y"]
                data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, filename = file_path)

                torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [39]:
dataset = MinCostDataset(root = "./data/")

100%|██████████| 34/34 [00:00<00:00, 28088.70it/s]


In [14]:
def dataset_information(dataset):
    print(dataset)
    print(f"num classes: {dataset.num_classes}")
    print(f"num features: {dataset.num_features}")
    print(f"first graph: {dataset[0]}")

dataset_information(dataset)

100%|██████████| 134/134 [00:00<00:00, 60040.25it/s]


MinCostDataset(132)


100%|██████████| 134/134 [00:00<00:00, 61980.23it/s]


num classes: 128


100%|██████████| 134/134 [00:00<00:00, 61277.45it/s]


num features: 1


100%|██████████| 134/134 [00:00<00:00, 55324.02it/s]


first graph: Data(x=[9559, 1], edge_index=[2, 29682], edge_attr=[29682, 2], y=[1, 1], filename='data/raw/road_flow_01_DC_a.txt')


In [63]:
from torch_geometric.nn import NNConv
import torch.nn.functional as F
import numpy as np

class CBN(torch.nn.Module):
    #TODO cite the colab
    def __init__(self, input_dim, output_dim, edge_feature_dim, args):
        super(CBN, self).__init__()

        hidden_dim = args.hidden_dim
        num_layers = args.num_layers
        dropout = args.dropout

        if num_layers > 1:
            conv_modules = [NNConv(input_dim, hidden_dim, nn.Linear(edge_feature_dim, input_dim * hidden_dim))]
            conv_modules.extend([NNConv(hidden_dim, hidden_dim, nn.Linear(edge_feature_dim, hidden_dim * hidden_dim)) for _ in range(num_layers - 2)])
            conv_modules.append(NNConv(hidden_dim, output_dim, nn.Linear(edge_feature_dim, hidden_dim * output_dim)))

            self.convs = nn.ModuleList(conv_modules)
        else:
            self.convs = nn.ModuleList([NNConv(input_dim, output_dim, nn.Linear(edge_feature_dim, input_dim * output_dim))])

        self.bns = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for _ in range(num_layers - 1)])

        # self.post_mp = nn.Linear(hidden_dim, 1)

        self.num_layers = num_layers

        # Probability of an element getting zeroed
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()
        # self.post_mp.reset_parameters()

    def forward(self, x, edge_index, edge_attr):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index, edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, self.dropout, self.training)
        x = self.convs[-1](x, edge_index, edge_attr)
        x = F.relu(x)
        # x = self.post_mp(x)

        return x

    def dual_value(N, p):
        return np.sum([p[i] * N.b[i] for i in N.V]) + np.sum([N.u[e] * max(0, p[e[1]] - p[e[0]] - N.c[e]) for e in N.E])

    # def loss(self, pred, label, x, edge_index, edge_attr):
    #     # edge_attr[0] is capacity, edge_attr[1] is cost
    #     print(pred.shape)
    #     print(edge_index[0].shape)
    #     print(pred[edge_index[1]].shape)
    #     print(edge_attr[:, 1].shape)
    #     reduced_cost = pred[edge_index[1]].squeeze() - pred[edge_index[0]].squeeze() - edge_attr[:, 1]
    #     print(reduced_cost.shape)
    #     return label - torch.dot(pred.squeeze(), x.squeeze()) - torch.dot(edge_attr[:, 0], F.relu(reduced_cost))

In [68]:
class DualLoss(nn.Module):
    def __init__(self):
        super(DualLoss, self).__init__()

    def forward(self, pred, label, x, edge_index, edge_attr):
        # edge_attr[0] is capacity, edge_attr[1] is cost
        #TODO is negative for the moment but if you switch the sign before the second dot product it's always positive ._.
        reduced_cost = pred[edge_index[1]].squeeze() - pred[edge_index[0]].squeeze() - edge_attr[:, 1]
        print(label, torch.dot(pred.squeeze(), x.squeeze()) - torch.dot(edge_attr[:, 0], F.relu(reduced_cost)))
        return label - torch.dot(pred.squeeze(), x.squeeze()) + torch.dot(edge_attr[:, 0], F.relu(reduced_cost))

In [43]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [44]:
import torch.optim as optim

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [81]:
args = {
    'num_layers': 5, 
    'batch_size': 32, 
    'hidden_dim': 64, 
    'dropout': 0.5, 
    'epochs': 500, 
    'opt': 'adam', 
    'opt_scheduler': 'none', 
    'opt_restart': 0, 
    'weight_decay': 5e-3, 
    'lr': 0.01
}
args = objectview(args)
model = CBN(1, 1, 2, args)
loss_fn = DualLoss()
data = dataset[40]


scheduler, opt = build_optimizer(args, model.parameters())
for i in range(250):
    model.train()
    opt.zero_grad()
    pred = model(data.x, data.edge_index, data.edge_attr)
    loss = loss_fn(pred, data.y, data.x, data.edge_index, data.edge_attr)
    print(f"loss: {loss.item()}")
    loss.backward()
    opt.step()
print(pred)

100%|██████████| 134/134 [00:00<00:00, 107814.45it/s]


tensor([[944]]) tensor(-16014.3760, grad_fn=<SubBackward0>)
loss: 16958.376953125
tensor([[944]]) tensor(-14356.9473, grad_fn=<SubBackward0>)
loss: 15300.947265625
tensor([[944]]) tensor(-1138.1403, grad_fn=<SubBackward0>)
loss: 2082.140380859375
tensor([[944]]) tensor(16.6134, grad_fn=<SubBackward0>)
loss: 927.3865356445312
tensor([[944]]) tensor(0., grad_fn=<SubBackward0>)
loss: 944.0
tensor([[944]]) tensor(6.9020, grad_fn=<SubBackward0>)
loss: 937.0980224609375
tensor([[944]]) tensor(103.3330, grad_fn=<SubBackward0>)
loss: 840.6669921875
tensor([[944]]) tensor(0., grad_fn=<SubBackward0>)
loss: 944.0
tensor([[944]]) tensor(14.4042, grad_fn=<SubBackward0>)
loss: 929.5958251953125
tensor([[944]]) tensor(8.6901, grad_fn=<SubBackward0>)
loss: 935.3098754882812
tensor([[944]]) tensor(11.9000, grad_fn=<SubBackward0>)
loss: 932.0999755859375
tensor([[944]]) tensor(1.2791, grad_fn=<SubBackward0>)
loss: 942.7208251953125
tensor([[944]]) tensor(20.6822, grad_fn=<SubBackward0>)
loss: 923.317810

In [91]:
p = pred.detach().numpy().flatten()
p

array([482.83713, 468.93777,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     , 164.39926,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ], dtype=float32)

In [ ]:
import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(dataset, args):

    print("Node task. test set size:", np.sum(dataset[0]['test_mask'].numpy()))
    print()
    test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)

    # build model
    output_dim = 1 # we predict scalar potential values for each vertex
    model = CBN(dataset.num_node_features, output_dim, dataset.num_edge_features, args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        for batch in loader:
            print(f"BATCH {batch}")
            opt.zero_grad()
            pred = model(batch)
            label = batch.y
            print(f"BATCH y: {batch.y.shape}")
            # pred = pred[batch.train_mask]
            # label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        losses.append(total_loss)

        if epoch % 10 == 0:
          test_acc = test(test_loader, model)
          test_accs.append(test_acc)
          if test_acc > best_acc:
            best_acc = test_acc
            best_model = copy.deepcopy(model)
        else:
          test_accs.append(test_accs[-1])

    return test_accs, losses, best_model, best_acc, test_loader

def test(loader, test_model, is_validation=False, save_model_preds=False, model_type=None):
    test_model.eval()

    correct = 0
    # Note that Cora is only one graph!
    for data in loader:
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            pred = test_model(data).max(dim=1)[1]
            label = data.y

        mask = data.val_mask if is_validation else data.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = label[mask]

        if save_model_preds:
          print ("Saving Model Predictions for Model Type", model_type)

          data = {}
          data['pred'] = pred.view(-1).cpu().detach().numpy()
          data['label'] = label.view(-1).cpu().detach().numpy()

          df = pd.DataFrame(data=data)
          # Save locally as csv
          df.to_csv('MinCostFlow-' + model_type + '.csv', sep=',', index=False)

        correct += pred.eq(label).sum().item()

    total = 0
    for data in loader.dataset:
        total += torch.sum(data.val_mask if is_validation else data.test_mask).item()

    return correct / total

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d